In [ ]:
!pip install nhanes pyjanitor
import nhanes.load
import janitor

In [ ]:
import pandas as pd
import sklearn.compose
import sklearn.preprocessing

In [ ]:
# unaVariableMas -> una_variable_mas
nhanes_raw_df = (
    nhanes.load.load_NHANES_data(year="2017-2018")
    .clean_names(
        case_type="snake"
    )
)
nhanes_raw_df.head(2)

In [4]:
nhanes_df = (
    nhanes_raw_df.select_columns(
        "general_health_condition",
        "gender",
        "age_in_years_at_screening"
    ).rename_columns(
        {
           "age_in_years_at_screening" : "age"
        }
    )
)
nhanes_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `jn.select` instead.
  return method(self._obj, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


,general_health_condition,gender,age
SEQN,,,
93703.0,NaN,Female,2.0
93704.0,NaN,Male,2.0
93705.0,Good,Female,66.0
93706.0,Very good,Male,18.0
93707.0,Good,Male,13.0
...,...,...,...
102952.0,Very good,Female,70.0
102953.0,Fair or,Male,42.0
102954.0,Good,Female,41.0


# One-Hot Encoding

In [ ]:
nhanes_df.general_health_condition.value_counts()

In [ ]:
transformer = sklearn.compose.make_column_transformer(
    (sklearn.preprocessing.OrdinalEncoder(), ["gender"]),
    (sklearn.preprocessing.OneHotEncoder(), ["general_health_condition"]),
    remainder="passthrough"
)
transformer

In [15]:
nhanes_transformed_df = (
    pd.DataFrame(
        transformer.fit_transform(nhanes_df),
        columns = transformer.get_feature_names_out(),
        index = nhanes_df.index
    )
    .rename_columns(
        function = lambda x: x.removeprefix("ordinalencoder__")
    )
    .rename_columns(
        function = lambda x: x.removeprefix("onehotencoder__")
    )
    .rename_columns(
        function = lambda x: x.removeprefix("remainder__")
    )
)
nhanes_transformed_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


,gender,general_health_condition_Excellent,general_health_condition_Fair or,general_health_condition_Good,general_health_condition_Poor?,general_health_condition_Very good,general_health_condition_nan,age
SEQN,,,,,,,,
93703.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
93704.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
93705.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,66.0
93706.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
93707.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...
102952.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,70.0
102953.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,42.0
102954.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0
